In [1]:
# !pip install typing-extensions==4.8.0
# !pip install fastapi
# !pip install uvicorn
# !pip install pydantic
# !pip install requests
# !pip install pypi-json
# !pip install pyngrok
# !pip install nest-asyncio

In [10]:
# !pip install datasets evaluate 'transformers[sentencepiece]'
# # !pip install protobuf==3.20.3
# ## !pip install --upgrade protobuf
# # !pip install keras==2.15.0
# ## !pip install --upgrade keras
# # !pip install --upgrade tensorflow
# !pip install --upgrade transformers
# !pip install faiss-cpu

  Using cached protobuf-3.20.2-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/20/0a/739426a81f7635b422fbe6cb8d1d99d1235579a6ac8024c13d743efa6847/transformers-4.36.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.3 MB/s eta 0:00:00
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/05/09/1945ca6ba3ad8ad6e2872ba6

In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import pandas as pd
import joblib
from transformers import AutoTokenizer, TFAutoModel

In [2]:
app = FastAPI()

In [3]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [4]:
class model_input(BaseModel):
    
    text : str

In [5]:
# import transformers
# transformers.__version__

'4.36.2'

In [5]:
loaded_dataset = joblib.load("embeddings_dataset.joblib")

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [6]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [7]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [8]:
@app.post('/soc_prediction')
def soc_pred(input_parameters: model_input):
    
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)
    
    text = input_dictionary['text']

    #Embed data
    text_embedding = get_embeddings([text]).numpy()

    scores, samples = loaded_dataset.get_nearest_examples(
        "embeddings", text_embedding, k=5
    )

    samples_df = pd.DataFrame.from_dict(samples)
    samples_df["score"] = scores
    samples_df.sort_values("score", ascending=False, inplace=True)

    return [{'label': row.label, 'score': row.score, 'text': row.text} for _, row in samples_df.iterrows()]

In [9]:
# question = "DUTIES include: apply caulk, sealants, or other agents to installed surfaces; apply grout between joints of bricks or tiles, using grouting trowels; arrange or store materials, machines, tools, or equipment; clean installation surfaces, equipment, tools, work sites, or storage areas, using water, chemical solutions, oxygen lances, or polishing machines; correct surface imperfections or fill chipped, cracked, or broken bricks or tiles, using fillers, adhesives, or grouting materials; cut materials to specified sizes for installation, using power saws or tile cutters; erect scaffolding or other installation structures; locate and supply materials to masons for installation; mix mortar, plaster, and grout, manually or using machines, according to standard formulas; modify material moving, mixing, grouting, grinding, polishing, or cleaning procedures, according to installation or material requirements; move or position materials such as marble slabs, assisting team members using cranes, hoists, or dollies; provide assistance in the preparation, installation, repair, or rebuilding of tile, brick, or stone surfaces; remove damaged tile, brick, or mortar, and clean or prepare surfaces, using pliers, hammers, chisels, drills, wire brushes, or metal wire anchors; remove excess grout or residue from tile or brick joints, using sponges or trowels; transport materials, tools, or machines to installation sites, manually or using conveyance equipment."
# question_embedding = get_embeddings([question]).numpy()
# question_embedding.shape

# scores, samples = embeddings_dataset.get_nearest_examples(
#     "embeddings", question_embedding, k=5
# )

# samples_df = pd.DataFrame.from_dict(samples)
# samples_df["scores"] = scores
# samples_df.sort_values("scores", ascending=False, inplace=True)

In [10]:
# for _, row in samples_df.iterrows():
#     print(f"SOC: {row.label}")
#     print(f"SCORE: {row.scores}")
#     print(f"TEXT: {row.text}")
#     print("=" * 50)
#     print()

In [11]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

t=2023-12-20T15:26:48-0500 lvl=eror msg="failed to reconnect session" obj=tunnels.session obj=csess id=96b12cd6c9dd err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"

ERROR:  authentication failed: Usage of ngrok requires a verified account and authtoken.
ERROR:  
ERROR:  Sign up for an account: https://dashboard.ngrok.com/signup
ERROR:  Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken
ERROR:  
ERROR:  ERR_NGROK_4018
ERROR:  


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.